**Ensemble Learning**
* **Konsep Dasar**: Ide utamanya adalah "kebijaksanaan banyak orang" (*wisdom of the crowd*). Jika Anda mengajukan pertanyaan kompleks kepada ribuan orang acak dan menggabungkan jawaban mereka, jawaban gabungan ini seringkali lebih baik daripada jawaban seorang ahli. Demikian pula, menggabungkan prediksi dari sekelompok prediktor (*ensemble*) akan menghasilkan prediksi yang lebih baik.
* **Penggunaan**: Metode *Ensemble* sering digunakan menjelang akhir proyek *Machine Learning* untuk menggabungkan beberapa prediktor yang sudah baik menjadi prediktor yang lebih unggul. Metode ini seringkali menjadi bagian dari solusi pemenang dalam kompetisi *Machine Learning*.
* **Metode Populer**: Bab ini membahas metode *Ensemble* yang paling populer, termasuk *bagging*, *boosting*, dan *stacking*, serta *Random Forests*.

**Voting Classifiers**
* **Konsep**: Cara sederhana untuk membuat pengklasifikasi yang lebih baik adalah dengan menggabungkan prediksi dari beberapa pengklasifikasi yang sudah dilatih (misalnya, *Logistic Regression*, *SVM*, *Random Forest*, *K-Nearest Neighbors*) dan memprediksi kelas yang mendapatkan suara terbanyak.
* **Hard Voting Classifier**: Pengklasifikasi yang menggunakan suara mayoritas ini disebut *hard voting classifier*.
* **Keunggulan**: Voting classifier seringkali mencapai akurasi yang lebih tinggi daripada pengklasifikasi individu terbaik dalam *ensemble*, bahkan jika setiap pengklasifikasi adalah *weak learner* (sedikit lebih baik dari tebakan acak), asalkan ada cukup banyak *weak learner* dan mereka cukup beragam.
* **Diversitas**: Agar metode *Ensemble* bekerja paling baik, prediktor harus seindependen mungkin satu sama lain. Salah satu cara untuk mencapai prediktor yang beragam adalah dengan melatihnya menggunakan algoritma yang sangat berbeda, yang meningkatkan kemungkinan mereka membuat jenis kesalahan yang berbeda dan meningkatkan akurasi *ensemble*.
* **Implementasi di Scikit-Learn (Hard Voting)**:
    ```python
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.ensemble import VotingClassifier
    from sklearn.linear_model import LogisticRegression
    from sklearn.svm import SVC

    log_clf = LogisticRegression()
    rnd_clf = RandomForestClassifier()
    svm_clf = SVC()

    voting_clf = VotingClassifier(
        estimators=[('lr', log_clf), ('rf', rnd_clf), ('svc', svm_clf)],
        voting='hard')

    voting_clf.fit(X_train, y_train)
    ```
    Kode ini melatih *VotingClassifier* yang terdiri dari tiga pengklasifikasi yang berbeda. Hasilnya menunjukkan bahwa *VotingClassifier* sedikit mengungguli pengklasifikasi individu.
* **Soft Voting**: Jika semua pengklasifikasi dapat mengestimasi probabilitas kelas (memiliki metode `predict_proba()`), Scikit-Learn dapat memprediksi kelas dengan probabilitas kelas tertinggi, dirata-ratakan di semua pengklasifikasi individu. Ini disebut *soft voting* dan seringkali mencapai kinerja yang lebih tinggi daripada *hard voting* karena memberikan bobot lebih pada suara yang sangat percaya diri. Untuk menggunakannya, set `voting="soft"` dan pastikan pengklasifikasi dapat mengestimasi probabilitas kelas (misalnya, untuk `SVC`, set `probability=True`).

**Bagging dan Pasting**
* **Konsep**: Cara lain untuk mendapatkan prediktor yang beragam adalah dengan menggunakan algoritma pelatihan yang sama untuk setiap prediktor, tetapi melatihnya pada subset acak yang berbeda dari *training set*.
    * **Bagging**: Jika pengambilan sampel dilakukan dengan penggantian (*with replacement*), metode ini disebut *bagging* (singkatan dari *bootstrap aggregating*).
    * **Pasting**: Jika pengambilan sampel dilakukan tanpa penggantian (*without replacement*), metode ini disebut *pasting*.
* **Perbedaan Utama**: Keduanya memungkinkan instansi pelatihan diambil sampelnya beberapa kali di berbagai prediktor, tetapi hanya *bagging* yang memungkinkan instansi pelatihan diambil sampelnya beberapa kali untuk prediktor yang sama.
* **Agregasi**: Setelah semua prediktor dilatih, *ensemble* membuat prediksi dengan menggabungkan prediksi semua prediktor. Fungsi agregasi biasanya adalah mode statistik (prediksi yang paling sering) untuk klasifikasi, atau rata-rata untuk regresi.
* **Bias dan Varians**: Setiap prediktor individu memiliki bias yang lebih tinggi daripada jika dilatih pada *training set* asli, tetapi agregasi mengurangi bias dan varians. Secara umum, *ensemble* memiliki bias yang serupa tetapi varians yang lebih rendah daripada prediktor tunggal yang dilatih pada *training set* asli.
* **Paralelisasi**: Prediktor dapat dilatih secara paralel, dan prediksi juga dapat dibuat secara paralel. Ini membuat *bagging* dan *pasting* sangat populer karena skalabilitasnya yang baik.
* **Implementasi di Scikit-Learn**: Scikit-Learn menyediakan kelas `BaggingClassifier` (dan `BaggingRegressor`).
    ```python
    from sklearn.ensemble import BaggingClassifier
    from sklearn.tree import DecisionTreeClassifier

    bag_clf = BaggingClassifier(
        DecisionTreeClassifier(), n_estimators=500,
        max_samples=100, bootstrap=True, n_jobs=-1)
    bag_clf.fit(X_train, y_train)
    y_pred = bag_clf.predict(X_test)
    ```
    Kode ini melatih *ensemble* 500 *Decision Tree classifier* menggunakan *bagging* (`bootstrap=True`). Jika `bootstrap=False`, itu akan menjadi *pasting*. `n_jobs=-1` menggunakan semua *core* CPU yang tersedia.
* **Bagging vs. Pasting**: *Bootstrapping* (*bagging*) memperkenalkan lebih banyak keragaman dalam subset pelatihan, menghasilkan bias yang sedikit lebih tinggi tetapi prediktor yang kurang berkorelasi, sehingga varians *ensemble* berkurang. Secara keseluruhan, *bagging* sering menghasilkan model yang lebih baik dan umumnya lebih disukai.

**Out-of-Bag Evaluation**
* **Konsep**: Dengan *bagging*, beberapa instansi pelatihan dapat diambil sampelnya berkali-kali, sementara yang lain mungkin tidak sama sekali. Rata-rata, sekitar 37% instansi pelatihan tidak diambil sampelnya untuk setiap prediktor; ini disebut instansi *out-of-bag* (oob).
* **Manfaat**: Karena prediktor tidak pernah melihat instansi oob selama pelatihan, mereka dapat dievaluasi pada instansi ini tanpa memerlukan *validation set* terpisah. *Ensemble* itu sendiri dapat dievaluasi dengan merata-ratakan evaluasi oob dari setiap prediktor.
* **Implementasi di Scikit-Learn**: Set `oob_score=True` saat membuat `BaggingClassifier`.
    ```python
    bag_clf = BaggingClassifier(
        DecisionTreeClassifier(), n_estimators=500,
        bootstrap=True, n_jobs=-1, oob_score=True)
    bag_clf.fit(X_train, y_train)
    print(bag_clf.oob_score_)
    ```
    `oob_score_` akan memberikan skor akurasi berdasarkan evaluasi oob.

**Random Patches dan Random Subspaces**
* **Sampling Fitur**: Kelas `BaggingClassifier` juga mendukung pengambilan sampel fitur melalui `max_features` dan `bootstrap_features`. Setiap prediktor akan dilatih pada subset acak dari fitur input.
* **Random Patches**: Mengambil sampel instansi pelatihan dan fitur.
* **Random Subspaces**: Menjaga semua instansi pelatihan (`bootstrap=False` dan `max_samples=1.0`) tetapi mengambil sampel fitur.
* **Manfaat**: Sampling fitur menghasilkan keragaman prediktor yang lebih besar, menukar sedikit bias dengan varians yang lebih rendah. Ini sangat berguna untuk input berdimensi tinggi.

**Random Forests**
* **Definisi**: *Random Forest* adalah *ensemble* *Decision Tree* yang umumnya dilatih menggunakan metode *bagging* (atau kadang *pasting*), biasanya dengan `max_samples` diatur ke ukuran *training set*.
* **Implementasi di Scikit-Learn**: Daripada menggunakan `BaggingClassifier` dengan `DecisionTreeClassifier`, lebih mudah dan lebih optimal menggunakan kelas `RandomForestClassifier` (atau `RandomForestRegressor`).
    ```python
    from sklearn.ensemble import RandomForestClassifier

    rnd_clf = RandomForestClassifier(n_estimators=500, max_leaf_nodes=16, n_jobs=-1)
    rnd_clf.fit(X_train, y_train)
    y_pred_rf = rnd_clf.predict(X_test)
    ```
* **Ekstra Randomness**: Algoritma *Random Forest* memperkenalkan keacakan ekstra saat menumbuhkan pohon; alih-alih mencari fitur terbaik saat membagi *node*, ia mencari fitur terbaik di antara subset fitur acak. Ini menghasilkan keragaman pohon yang lebih besar, yang menukar bias yang lebih tinggi dengan varians yang lebih rendah, menghasilkan model yang umumnya lebih baik secara keseluruhan.

**Extra-Trees**
* **Konsep**: *Extremely Randomized Trees ensemble* (atau *Extra-Trees*) membuat pohon lebih acak dengan menggunakan ambang acak untuk setiap fitur daripada mencari ambang terbaik.
* **Manfaat**: Teknik ini juga menukar lebih banyak bias dengan varians yang lebih rendah. Selain itu, *Extra-Trees* jauh lebih cepat dilatih daripada *Random Forests* karena tidak perlu mencari ambang terbaik.
* **Implementasi di Scikit-Learn**: Gunakan kelas `ExtraTreesClassifier` atau `ExtraTreesRegressor`. API-nya identik dengan kelas `RandomForestClassifier`.
* **Perbandingan**: Sulit untuk menentukan sebelumnya apakah `RandomForestClassifier` atau `ExtraTreesClassifier` akan berkinerja lebih baik; cara terbaik adalah mencoba keduanya dan membandingkannya menggunakan validasi silang.

**Feature Importance**
* **Konsep**: *Random Forests* memudahkan pengukuran kepentingan relatif setiap fitur. Scikit-Learn mengukur kepentingan fitur dengan melihat seberapa banyak *node* pohon yang menggunakan fitur tersebut mengurangi *impurity* rata-rata (di semua pohon dalam *forest*).
* **Akses**: Skor ini diskalakan sehingga jumlah semua kepentingan sama dengan 1 dan dapat diakses melalui variabel `feature_importances_`.
* **Contoh (Iris Dataset)**:
    ```python
    from sklearn.datasets import load_iris
    from sklearn.ensemble import RandomForestClassifier

    iris = load_iris()
    rnd_clf = RandomForestClassifier(n_estimators=500, n_jobs=-1)
    rnd_clf.fit(iris["data"], iris["target"])
    for name, score in zip(iris["feature_names"], rnd_clf.feature_importances_):
        print(name, score)
    ```
    Contoh ini menunjukkan bahwa *petal length* dan *petal width* adalah fitur yang paling penting dalam dataset *Iris*.
* **Kegunaan**: *Random Forests* sangat berguna untuk mendapatkan pemahaman cepat tentang fitur mana yang sebenarnya penting, terutama untuk pemilihan fitur.

**Boosting**
* **Konsep**: *Boosting* (awalnya disebut *hypothesis boosting*) adalah metode *Ensemble* apa pun yang dapat menggabungkan beberapa *weak learner* menjadi *strong learner*. Ide umumnya adalah melatih prediktor secara berurutan, dengan masing-masing mencoba mengoreksi pendahulunya.
* **Metode Populer**: *AdaBoost* dan *Gradient Boosting*.

**AdaBoost**
* **Konsep**: Prediktor baru mencoba mengoreksi pendahulunya dengan memberikan perhatian lebih pada instansi pelatihan yang *underfitted* oleh pendahulu. Ini membuat prediktor baru semakin fokus pada kasus-kasus sulit.
* **Cara Kerja**:
    1.  Algoritma melatih pengklasifikasi dasar (misalnya, *Decision Tree*) dan menggunakannya untuk membuat prediksi pada *training set*.
    2.  Kemudian meningkatkan bobot relatif dari instansi pelatihan yang salah diklasifikasikan.
    3.  Melatih pengklasifikasi kedua menggunakan bobot yang diperbarui, membuat prediksi lagi, memperbarui bobot instansi, dan seterusnya.
* **Persamaan dengan Gradient Descent**: Teknik pembelajaran sekuensial ini memiliki kemiripan dengan *Gradient Descent*, tetapi alih-alih mengubah parameter prediktor tunggal untuk meminimalkan fungsi biaya, *AdaBoost* menambahkan prediktor ke *ensemble*, secara bertahap membuatnya lebih baik.
* **Keterbatasan**: Pembelajaran sekuensial ini tidak dapat diparalelkan sepenuhnya, sehingga tidak skalabel sebaik *bagging* atau *pasting*.
* **Proses Algoritma**:
    * Bobot instansi $w^{(i)}$ awalnya diatur ke $1/m$.
    * Tingkat kesalahan tertimbang ($r_j$) dari prediktor ke-$j$ dihitung.
    * Bobot prediktor ($\alpha_j$) dihitung berdasarkan $r_j$. Semakin akurat prediktor, semakin tinggi bobotnya.
    * Bobot instansi diperbarui, meningkatkan bobot instansi yang salah diklasifikasikan.
    * Semua bobot instansi dinormalisasi.
    * Proses diulang hingga jumlah prediktor yang diinginkan tercapai atau prediktor sempurna ditemukan.
* **Prediksi**: *AdaBoost* menggabungkan prediksi semua prediktor dan membobotnya menggunakan bobot prediktor ($\alpha_j$). Kelas yang diprediksi adalah kelas yang menerima suara tertimbang mayoritas.
* **Implementasi di Scikit-Learn**: Scikit-Learn menggunakan versi *multiclass* dari *AdaBoost* yang disebut SAMME (atau SAMME.R jika prediktor dapat mengestimasi probabilitas kelas).
    ```python
    from sklearn.ensemble import AdaBoostClassifier
    from sklearn.tree import DecisionTreeClassifier

    ada_clf = AdaBoostClassifier(
        DecisionTreeClassifier(max_depth=1), n_estimators=200,
        algorithm="SAMME.R", learning_rate=0.5)
    ada_clf.fit(X_train, y_train)
    ```
    *Decision Stump* (`max_depth=1`) adalah *base estimator* default untuk `AdaBoostClassifier`. Jika *AdaBoost ensemble* *underfitting*, coba kurangi jumlah estimator atau regularisasi *base estimator* lebih kuat.

**Gradient Boosting**
* **Konsep**: Mirip dengan *AdaBoost*, *Gradient Boosting* juga menambahkan prediktor secara sekuensial, masing-masing mengoreksi pendahulunya. Namun, alih-alih mengubah bobot instansi, metode ini mencoba menyesuaikan prediktor baru dengan *residual errors* (kesalahan sisa) yang dibuat oleh prediktor sebelumnya.
* **Gradient Tree Boosting (GBRT)**: Menggunakan *Decision Trees* sebagai *base predictor* untuk regresi.
* **Cara Kerja (Contoh Regresi)**:
    1.  Latih `DecisionTreeRegressor` pertama pada *training set*.
    2.  Latih `DecisionTreeRegressor` kedua pada *residual errors* yang dibuat oleh prediktor pertama.
    3.  Latih `DecisionTreeRegressor` ketiga pada *residual errors* yang dibuat oleh prediktor kedua.
    4.  Prediksi baru dibuat dengan menjumlahkan prediksi dari semua pohon.
* **Implementasi di Scikit-Learn**: Gunakan kelas `GradientBoostingRegressor`.
    ```python
    from sklearn.ensemble import GradientBoostingRegressor

    gbrt = GradientBoostingRegressor(max_depth=2, n_estimators=3, learning_rate=1.0)
    gbrt.fit(X, y)
    ```
* **Hyperparameter `learning_rate`**: Mengatur kontribusi setiap pohon. Nilai yang rendah (misalnya, 0.1) akan membutuhkan lebih banyak pohon tetapi biasanya akan menggeneralisasi lebih baik (teknik regularisasi yang disebut *shrinkage*).
* **Early Stopping**: Digunakan untuk menemukan jumlah pohon yang optimal. Metode `staged_predict()` mengembalikan iterator di atas prediksi yang dibuat oleh *ensemble* pada setiap tahap pelatihan.
    ```python
    import numpy as np
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import mean_squared_error

    # Asumsi X, y sudah tersedia
    X_train, X_val, y_train, y_val = train_test_split(X, y)

    gbrt = GradientBoostingRegressor(max_depth=2, n_estimators=120)
    gbrt.fit(X_train, y_train)

    errors = [mean_squared_error(y_val, y_pred)
              for y_pred in gbrt.staged_predict(X_val)]
    bst_n_estimators = np.argmin(errors) + 1

    gbrt_best = GradientBoostingRegressor(max_depth=2, n_estimators=bst_n_estimators)
    gbrt_best.fit(X_train, y_train)
    ```
    *Early stopping* juga bisa diimplementasikan dengan menghentikan pelatihan lebih awal menggunakan `warm_start=True`.
* **Subsample (`Stochastic Gradient Boosting`)**: Hyperparameter `subsample` menentukan fraksi instansi pelatihan yang digunakan untuk melatih setiap pohon. Ini menukar bias yang lebih tinggi dengan varians yang lebih rendah dan mempercepat pelatihan.
* **XGBoost**: Implementasi *Gradient Boosting* yang sangat optimal tersedia di pustaka Python populer XGBoost (*Extreme Gradient Boosting*). XGBoost dirancang untuk sangat cepat, skalabel, dan portabel, dan sering menjadi komponen penting dalam kompetisi ML.
    ```python
    import xgboost

    xgb_reg = xgboost.XGBRegressor()
    xgb_reg.fit(X_train, y_train)
    y_pred = xgb_reg.predict(X_val)

    # Dengan early stopping otomatis
    xgb_reg.fit(X_train, y_train,
                eval_set=[(X_val, y_val)], early_stopping_rounds=2)
    ```

**Stacking**
* **Konsep**: *Stacking* (atau *stacked generalization*) didasarkan pada ide untuk melatih sebuah model (disebut *blender* atau *meta learner*) untuk menggabungkan prediksi dari semua prediktor dalam *ensemble*, alih-alih menggunakan fungsi trivial seperti *hard voting*.
* **Proses Pelatihan (menggunakan *hold-out set*)**:
    1.  *Training set* dibagi menjadi dua subset.
    2.  Subset pertama digunakan untuk melatih prediktor di lapisan pertama.
    3.  Prediktor lapisan pertama digunakan untuk membuat prediksi pada subset kedua (*held-out set*). Ini memastikan prediksi "bersih" karena prediktor tidak melihat instansi ini selama pelatihan.
    4.  Sebuah *new training set* dibuat menggunakan nilai prediksi ini sebagai fitur input dan mempertahankan nilai target aslinya.
    5.  *Blender* dilatih pada *new training set* ini, sehingga belajar memprediksi nilai target berdasarkan prediksi lapisan pertama.
* **Lapisan Multilayer**: Dimungkinkan untuk melatih beberapa *blender* yang berbeda atau bahkan membuat lapisan *blender* tambahan dengan membagi *training set* menjadi tiga subset atau lebih, di mana setiap subset digunakan untuk melatih lapisan berikutnya.
* **Prediksi**: Untuk membuat prediksi pada instansi baru, prediksi dibuat secara berurutan melalui setiap lapisan.
* **Implementasi**: Scikit-Learn tidak mendukung *stacking* secara langsung, tetapi dapat diimplementasikan secara manual. Pustaka sumber terbuka seperti DESlib juga dapat digunakan.